<a href="https://colab.research.google.com/github/steinhaug/stable-diffusion/blob/main/BLIP-2/AutoCaptioner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Connect runtime and mount Google Drive
#@markdown Check type of GPU and VRAM available.
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

from google.colab import drive
drive.mount('/content/drive')

Tesla V100-SXM2-16GB, 16384 MiB, 16150 MiB
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
def ret_directoryFileCount(dir_path):
    file_count = 0
    for path in os.listdir(dir_path):
        if os.path.isfile(os.path.join(dir_path, path)):
            file_count += 1
    return file_count

IMAGE_FOLDERS = '/content/drive/MyDrive/datasets/PinupFiles'

In [3]:
from zipfile import ZipFile
import os

for item in os.listdir(IMAGE_FOLDERS):
    zips = os.path.join(IMAGE_FOLDERS, item)
    print(f'Extracting: {item}')
    with ZipFile(zips, 'r') as zObject:
        zObject.extractall(path=IMAGE_FOLDERS)

Extracting: Alice-Goodwin.zip
Extracting: Beth-Lily.zip
Extracting: Holly-Peers.zip
Extracting: Jordan-Carver.zip
Extracting: Melissa-Debling.zip
Extracting: Sammy-Braddy.zip
Extracting: Sophie-Howard.zip
Extracting: Stacey-Poole.zip
Extracting: Tessa-Fowler.zip


In [4]:
from IPython.display import clear_output
import sys
if 'google.colab' in sys.modules:
    !pip3 install salesforce-lavis
clear_output()
print('[1;32mDone! ✓')

Done! ✓


In [5]:
import torch
from PIL import Image
import requests
from lavis.models import load_model_and_preprocess
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

In [6]:
model, vis_processors, _ = load_model_and_preprocess(
    name="blip2_opt", model_type="pretrain_opt2.7b", is_eval=True, device=device
)
#model, vis_processors, _ = load_model_and_preprocess(
#    name="blip2_opt", model_type="caption_coco_opt2.7b", is_eval=True, device=device
#)
vis_processors.keys()

100%|██████████| 1.89G/1.89G [01:06<00:00, 30.5MB/s]


100%|██████████| 409M/409M [00:20<00:00, 21.2MB/s]


dict_keys(['train', 'eval'])

In [27]:
import os
def load_image(image_path, sizeDiv=8):
    raw_image = Image.open(f'{image_path}').convert('RGB')
    h, w = raw_image.size
    display(raw_image.resize(( int(h / sizeDiv) , int(w / sizeDiv) )))
    image = vis_processors["eval"](raw_image).unsqueeze(0).to(device)
    return image

def wrap_query(query):
    return "Question: " + query + " Answer:"

def write_the_file(path, data_string):
    if len(str(data_string)):
        with open(path, 'w+') as fw:
            fw.write(str(data_string))
    else:
        if os.path.exists(path):
            os.remove(path)
    return '';

#image = vis_processors["eval"](raw_image).unsqueeze(0).to(device)

In [29]:
def ret__longest_val(arr):
    curr_v = ''
    curr_x = 0
    while arr:
        val = arr.pop(0)
        if len(val) >= curr_x:
            curr_x = len(val)
            curr_v = val
    return curr_v

def auto__get_AI_caption(IMG_SRC):
    image = load_image(IMG_SRC)
    cap1 = ret__longest_val( model.generate({"image": image}, use_nucleus_sampling=False, num_captions=4) )
    cap2 = ret__longest_val( model.generate({"image": image}, use_nucleus_sampling=True, num_captions=4) )
    cap3 = model.generate({"image": image, "prompt": wrap_query('What is out of the ordinary with this image?')}).pop(0)
    return [cap1, cap2, cap3]

#caption = auto__get_AI_caption('/content/drive/MyDrive/datasets/PinupFiles/Beth-Lily/beth-lily-vol-1-set-1-1.jpg')
#write_the_file('/content/test.txt', caption)
#print(caption)

In [31]:
import os
for item_name in os.listdir(IMAGE_FOLDERS):
    item = os.path.join(IMAGE_FOLDERS, item_name)
    if os.path.isdir(item):
        print( f'{item_name}: {ret_directoryFileCount(item)}' )

Alice-Goodwin: 61
Beth-Lily: 307
Holly-Peers: 191
Jordan-Carver: 205
Melissa-Debling: 191
Sammy-Braddy: 162
Sophie-Howard: 33
Stacey-Poole: 534
Tessa-Fowler: 807


In [ ]:
import os
import random
import json

def auto__create_caption(IMG_SRC, CAPTION_SRC):

    if os.path.isfile(CAPTION_SRC):
        return 'Exists, continue'

    caption_array = auto__get_AI_caption(IMG_SRC)
    write_the_file(CAPTION_SRC, json.dumps(caption_array))
    return json.dumps(caption_array)

for IMAGE_FOLDER in os.listdir(IMAGE_FOLDERS):
    FOLDER = os.path.join(IMAGE_FOLDERS, IMAGE_FOLDER)
    if os.path.isdir(FOLDER):
        os.makedirs(FOLDER + '_captions', exist_ok=True)
        for FILE_NAME in os.listdir(FOLDER):
            IMG_SRC = os.path.join(FOLDER, FILE_NAME)
            if os.path.isfile(IMG_SRC):
                f_name, f_ext = os.path.splitext(FILE_NAME)
                CAPTION_SRC = FOLDER + f'_captions/{f_name}.txt'
                caption = auto__create_caption(IMG_SRC, CAPTION_SRC)
                print(f'{f_name}: {caption}')
#    if IMAGE_FOLDER == 'Holly-Peers':
#        print('Found Holly-Peers')
#        break

In [36]:
import json

arr = ['aaa','bbb','ccc']
print( json.dumps(arr) )


#print( arr.toString() )
#.encode('utf-8')

#print( ", ".join(arr)  )

b'["aaa", "bbb", "ccc"]'
